In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

1. Reading the data in python
1. Defining the problem statement
1. Identifying the Target variable
1. Looking at the distribution of Target variable
1. Basic Data exploration
1. Rejecting useless columns
1. Visual Exploratory Data Analysis for data distribution (Histogram and Barcharts)
1. Feature Selection based on data distribution
1. Outlier treatment
1. Missing Values treatment
1. Visual correlation analysis
1. Statistical correlation analysis (Feature Selection)
1. Converting data to numeric for ML

1. **READING THE DATA**

In [ ]:
train_data= pd.read_csv('/kaggle/input/playground-series-s3e4/train.csv')
train_data.head()

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
print('Shape of data before deleting duplicates',train_data.shape)
train_data= train_data.drop_duplicates()
print('Shape of data after deleting duplicates',train_data.shape)

2.   **DEFINING THE PROBLEM TATEMENT**

Target Variable = class
Predictor Variable = time, amount etc

In [ ]:
%matplotlib inline
GroupData = train_data.groupby('Class').size()
GroupData.plot(kind='bar')

In [ ]:
train_data['Class'].value_counts()

In [ ]:
train_data[['Time','Amount','Class']].describe()

In [ ]:
train_data.info()

In [ ]:
train_data.nunique()

In [ ]:
def indices_of_outliers(x):
    q1,q3 =np.percentile(x, [25,75])
    iqr=q3-q1
    lower_bound= q1-(iqr*1.5)
    upper_bound= q3 +(iqr* 1.5)
    
    
    return np.where((x>upper_bound)| (x<lower_bound))

In [ ]:
indices_of_outliers(train_data['Amount'])

In [ ]:
len(indices_of_outliers(train_data['Amount'])[0])

In [ ]:
q1,q3 =np.percentile(train_data['Amount'], [25,75])

q1,q3

In [ ]:
iqr=q3-q1
lower_bound= q1-(iqr*1.5)
upper_bound= q3 +(iqr* 1.5)

In [ ]:
lower_bound,upper_bound

In [ ]:
len(train_data)

In [ ]:
len(indices_of_outliers(train_data['Amount'])[0])/len(train_data)

In [ ]:
train_data.hist(['Amount'],figsize=(10,5))

In [ ]:

train_data['Amount'][train_data['Amount']<1000].sort_values(ascending=False)

In [ ]:
train_data['Amount'][train_data['Amount']>1000]=999.95
train_data.hist(['Amount'],figsize=(10,5))

In [ ]:
train_data['Amount'][train_data['Amount']<600].sort_values(ascending=False)

In [ ]:
train_data['Amount'][train_data['Amount']>600]=599.99
train_data.hist(['Amount'],figsize=(10,5))

In [ ]:
train_data.hist(['V1'],figsize=(5,5))

In [ ]:
train_data.isnull().sum()

In [ ]:
from matplotlib import pyplot as plt
plt.figure(figsize=(7,4))
fig_data = train_data.corr()['Class'][:-1].abs().sort_values().plot(kind='bar', title='Highly correlated features with Class')

In [ ]:
train_data.head()

In [ ]:
def FunctionAnova(inpData, TargetVariable, ContinuousPredictorList):
    from scipy.stats import f_oneway

    # Creating an empty list of final selected predictors
    SelectedPredictors=[]
    
    print('##### ANOVA Results ##### \n')
    for predictor in ContinuousPredictorList:
        CategoryGroupLists=inpData.groupby(TargetVariable)[predictor].apply(list)
        AnovaResults = f_oneway(*CategoryGroupLists)
        
        # If the ANOVA P-Value is <0.05, that means we reject H0
        if (AnovaResults[1] < 0.05):
            print(predictor, 'is correlated with', TargetVariable, '| P-Value:', AnovaResults[1])
            SelectedPredictors.append(predictor)
        else:
            print(predictor, 'is NOT correlated with', TargetVariable, '| P-Value:', AnovaResults[1])
    
    return(SelectedPredictors)
    

In [ ]:
# Calling the function to check which categorical variables are correlated with target
ContinuousVariables=['V1','V2','V3','V4','V5','V6','V7','V8','V9','V10','V11','V12'
                    ,'V13','V14','V15','V16','V17','V18','V19','V20','V21','V22',
                    'V23','V24','V25','V26','V27','V28','Time','Amount']
FunctionAnova(inpData=train_data, TargetVariable='Class', ContinuousPredictorList=ContinuousVariables)

**V13,V22,V25,V27,AMOUNT NOT CORRELATED**

In [ ]:
train_data.drop(['V13','V22','V25','V27'],axis=1, inplace=True)

In [ ]:
predictors =['V1','V2','V3','V4','V5','V6','V7','V8','V9','V10','V11','V12'
                    ,'V14','V15','V16','V17','V18','V19','V20','V21',
                    'V23','V24','V26','V28','Time','Amount']
TargetVariable='Class'
X=train_data[predictors].values
y=train_data[TargetVariable].values

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
test_data = pd.read_csv('/kaggle/input/playground-series-s3e4/test.csv')
test_data.head()

In [ ]:
test_data.drop(['V13','V22','V25','V27'],axis=1, inplace=True)

In [ ]:
test_data.info()
test_data.shape

In [ ]:
test_data.hist(['Amount'],figsize=(10,5))


In [ ]:

test_data['Amount'][test_data['Amount']<1000].sort_values(ascending=False)

In [ ]:
train_data['Amount'][train_data['Amount']>1000]=996
train_data.hist(['Amount'],figsize=(10,5))

In [ ]:
test_data.isnull().sum()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore')
X_train , X_test , y_train, y_test = train_test_split(X , y , train_size = 0.8 , random_state = 2)

In [ ]:
y_train.shape

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()

model.fit(X_train,y_train)
y_pred = model.predict(X_test)

accuracy_score(y_test, y_pred)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

In [ ]:
y_pred

In [ ]:
data=pd.DataFrame(y_pred,columns=['Class'])
data1=pd.DataFrame(test_data.id)
result=pd.concat([data1,data],axis=1)
result

In [ ]:
result.to_csv('Submission.csv',index=False)
result

In [ ]:
result =result.reset_index(drop=True)
result